In [18]:
import pandas as pd
import os
import pickle
import numpy as np


In [19]:
def improved_preprocess_without_scaling(df, feature_columns, id_columns, categorical_columns, 
                                     categorical_modes, stats=None, random_state=42):
    """
    Helper function to preprocess data without scaling using improved method
    Now uses normal distribution sampling instead of backward filling
    """
    import numpy as np
    
    # Set random seed for reproducibility
    np.random.seed(random_state)
    
    # Make sure categorical columns don't have missing values first
    df_processed = df.copy()
    
    for col in categorical_columns:
        if col in df_processed.columns and df_processed[col].isna().sum() > 0:
            # Fill categorical missing values with the most frequent value from training
            df_processed[col].fillna(categorical_modes[col], inplace=True)
    
    # Step 1: Sort data by patient and time to ensure correct filling
    df_sorted = df_processed.sort_values(['PatientID', 'Hours'])
    df_processed = df_sorted.copy()
    
    # Apply forward filling within each patient group
    print("Applying forward filling...")
    for patient_id in df_processed['PatientID'].unique():
        patient_mask = df_processed['PatientID'] == patient_id
        df_processed.loc[patient_mask, feature_columns] = df_processed.loc[patient_mask, feature_columns].ffill()
    
    print(f"Missing values after forward filling: {df_processed[feature_columns].isna().sum().sum()}")
    
    # Instead of backward filling, sample from normal distribution for remaining NAs
    print("Sampling from normal distribution for remaining NAs...")
    
    # Get remaining nulls count before sampling
    remaining_nulls_before = df_processed[feature_columns].isna().sum().sum()
    
    # Define non-negative features
    non_negative_features = ['Glucose', 'BUN', 'Creatinine', 'AST', 'ALT', 'ALP', 'Bilirubin', 
                           'Troponin', 'Lactate', 'PaO2', 'Urine', 'WBC', 'Platelets', 
                           'O2Sat', 'GCS', 'Age']
    
    for col in feature_columns:
        # Check if we have nulls for this column
        null_mask = df_processed[col].isna()
        num_nulls = null_mask.sum()
        
        if num_nulls > 0:
            # If we have stats for normal distribution parameters
            if stats is not None and col in stats:
                mean_val = stats[col]['mean']
                std_val = stats[col]['std']
                
                # Add small epsilon to std to avoid division by zero
                epsilon = 1e-6
                
                # Generate samples from normal distribution
                samples = np.random.normal(mean_val, std_val + epsilon, size=num_nulls)
                
                # Clip at zero for non-negative features
                if col in non_negative_features:
                    samples = np.clip(samples, 0, None)
                
                # Assign sampled values to nulls
                df_processed.loc[null_mask, col] = samples
                
                print(f"Sampled {num_nulls} values for {col} from N({mean_val:.2f}, {std_val:.2f})")
            else:
                # Fallback to mean imputation if no stats available
                mean_val = df_processed[col].dropna().mean()
                if pd.isna(mean_val):  # If all values are NA (unlikely, but possible)
                    mean_val = 0
                df_processed[col].fillna(mean_val, inplace=True)
                print(f"Used mean imputation for {col}: {mean_val:.2f}")
    
    print(f"Missing values after sampling: {df_processed[feature_columns].isna().sum().sum()}")

    # Final check - any remaining nulls get mean imputation
    print("Imputing any remaining null values with column means...")
    for col in feature_columns:
        if df_processed[col].isna().sum() > 0:
            mean_val = df_processed[col].dropna().mean()
            if pd.isna(mean_val):  # If all values are NA (unlikely, but possible)
                mean_val = 0
            df_processed[col].fillna(mean_val, inplace=True)
    
    print(f"Missing values after complete imputation: {df_processed[feature_columns].isna().sum().sum()}")
    
    # Final check to catch any remaining missing values
    remaining_nulls = df_processed.isna().sum()
    if remaining_nulls.sum() > 0:
        print("WARNING: Remaining null values detected!")
        print(remaining_nulls[remaining_nulls > 0])
        print("Performing final fill with zeros...")
        df_processed.fillna(0, inplace=True)  # Final safety measure
    
    return df_processed

In [20]:



def fit_enhanced_preprocessor(train_data, output_folder='preprocessed_data'):
    """
    Fit preprocessing parameters with enhanced scaling on training data.
    Uses log transform, minmax, or robust scaling based on feature distributions.
    Also calculates normal distribution parameters for imputation.
    """
    import pandas as pd
    import numpy as np
    import os
    import pickle
    from sklearn.preprocessing import RobustScaler, MinMaxScaler
    
    # Read the training data if path is provided
    if isinstance(train_data, str):
        df = pd.read_csv(train_data)
    else:
        df = train_data.copy()
    
    print(f"Training data shape: {df.shape}")
    print(f"Missing values before preprocessing: {df.isna().sum().sum()}")
    
    # Define columns that we don't want to impute or scale
    id_columns = ['PatientID', 'Hours']
    categorical_columns = ['Gender', 'ICUType']
    
    # Get modes for categorical columns
    categorical_modes = {}
    for col in categorical_columns:
        if col in df.columns:
            categorical_modes[col] = df[col].mode()[0]
    
    # Group features by scaling type
    log_transform_features = ['Glucose', 'BUN', 'Creatinine', 'AST', 'ALT', 'ALP', 'Bilirubin', 
                             'Troponin', 'Lactate', 'PaO2', 'Urine', 'WBC', 'Platelets']
    
    minmax_features = ['pH', 'FiO2', 'O2Sat', 'GCS', 'Age']
    
    # Columns that need imputation and scaling
    feature_columns = [col for col in df.columns if col not in id_columns + categorical_columns]
    
    # All remaining features get robust scaling
    robust_scale_features = [col for col in feature_columns 
                            if col not in log_transform_features + minmax_features]
    
    # Filter to only include features present in the dataset
    log_transform_features = [col for col in log_transform_features if col in feature_columns]
    minmax_features = [col for col in minmax_features if col in feature_columns]
    robust_scale_features = [col for col in robust_scale_features if col in feature_columns]
    
    print(f"Log transform features: {len(log_transform_features)}")
    print(f"MinMax features: {len(minmax_features)}")
    print(f"Robust scale features: {len(robust_scale_features)}")
    
    # Helper function for preprocessing without scaling for fitting
    def preprocess_for_stats(df, feature_columns, id_columns, categorical_columns, categorical_modes):
        """Simple preprocessing to calculate distribution statistics"""
        df_processed = df.copy()
        
        # Apply categorical imputation
        for col in categorical_columns:
            if col in df_processed.columns and df_processed[col].isna().sum() > 0:
                df_processed[col].fillna(categorical_modes.get(col, df_processed[col].mode()[0]), inplace=True)
        
        # Apply forward and backward filling to get complete data for stats calculation
        for patient_id in df_processed['PatientID'].unique():
            patient_mask = df_processed['PatientID'] == patient_id
            df_processed.loc[patient_mask, feature_columns] = (
                df_processed.loc[patient_mask, feature_columns].ffill().bfill()
            )
        
        # Apply mean imputation for any remaining nulls
        for col in feature_columns:
            mean_val = df_processed[col].dropna().mean()
            if pd.isna(mean_val):
                mean_val = 0
            df_processed[col].fillna(mean_val, inplace=True)
        
        return df_processed
    
    # Get a complete dataset for calculating statistics
    df_for_stats = preprocess_for_stats(
        df,
        feature_columns,
        id_columns,
        categorical_columns,
        categorical_modes
    )
    
    # Calculate statistics from complete data
    stats = {}
    for col in feature_columns:
        values = df_for_stats[col]
        stats[col] = {
            'mean': values.mean(),
            'median': values.median(),
            'min': values.min(),
            'max': values.max(),
            'std': values.std(),
            'q1': values.quantile(0.25),
            'q3': values.quantile(0.75)
        }
    
    # Now preprocess training data using the normal distribution sampling
    df_train_processed = improved_preprocess_without_scaling(
        df,
        feature_columns,
        id_columns,
        categorical_columns,
        categorical_modes,
        stats=stats
    )
    
    # Fit different scalers
    # Robust scaler for features with outliers
    robust_scaler = RobustScaler()
    if robust_scale_features:
        robust_scaler.fit(df_train_processed[robust_scale_features])
    
    # MinMax scaler for bounded variables
    minmax_scaler = MinMaxScaler()
    if minmax_features:
        minmax_scaler.fit(df_train_processed[minmax_features])
    
    # Log transformation doesn't need fitting, but we'll store the epsilon value
    log_epsilon = 1e-6
    
    # Save the scalers and parameters
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Create a scaling info dictionary
    scaling_info = {
        'robust_features': robust_scale_features,
        'minmax_features': minmax_features,
        'log_features': log_transform_features,
        'feature_columns': feature_columns,
        'log_epsilon': log_epsilon
    }
    
    # Save all objects
    pickle.dump(robust_scaler, open(os.path.join(output_folder, 'robust_scaler.pkl'), 'wb'))
    pickle.dump(minmax_scaler, open(os.path.join(output_folder, 'minmax_scaler.pkl'), 'wb'))
    pickle.dump(scaling_info, open(os.path.join(output_folder, 'scaling_info.pkl'), 'wb'))
    pickle.dump(stats, open(os.path.join(output_folder, 'improved_imputation_stats.pkl'), 'wb'))
    pickle.dump(categorical_modes, open(os.path.join(output_folder, 'improved_categorical_modes.pkl'), 'wb'))
    
    print(f"Enhanced preprocessing parameters saved to: {output_folder}")
    
    return {
        'robust_scaler': robust_scaler,
        'minmax_scaler': minmax_scaler,
        'scaling_info': scaling_info,
        'stats': stats,
        'categorical_modes': categorical_modes
    }

In [21]:



def apply_enhanced_scaling(df, preprocessing_params):
    """
    Apply enhanced scaling to the preprocessed dataframe using log, minmax, or robust scaling
    """
    import numpy as np
    
    df_scaled = df.copy()
    
    # Extract parameters
    robust_scaler = preprocessing_params['robust_scaler']
    minmax_scaler = preprocessing_params['minmax_scaler']
    scaling_info = preprocessing_params['scaling_info']
    
    # Apply log transformation first
    for col in scaling_info['log_features']:
        if col in df.columns:
            # Add small epsilon to avoid log(0)
            epsilon = scaling_info.get('log_epsilon', 1e-6)
            # Clip negative values to epsilon
            df_scaled[col] = np.log1p(df[col].clip(lower=epsilon))
    
    # Apply robust scaling
    if scaling_info['robust_features']:
        df_scaled[scaling_info['robust_features']] = robust_scaler.transform(
            df[scaling_info['robust_features']]
        )
    
    # Apply minmax scaling
    if scaling_info['minmax_features']:
        df_scaled[scaling_info['minmax_features']] = minmax_scaler.transform(
            df[scaling_info['minmax_features']]
        )
    
    return df_scaled

In [22]:


def enhanced_preprocess_patient_data(data, preprocessing_params=None, output_folder='preprocessed_data', 
                                    output_filename=None, is_training=False, random_state=42):
    """
    Preprocess patient data with enhanced method using log, minmax, or robust scaling
    based on feature distributions. Now uses normal distribution sampling instead of backward filling.
    """
 
    
    # Read the data if path is provided
    if isinstance(data, str):
        df = pd.read_csv(data)
        # Generate a filename if needed
        if output_filename is None:
            base_filename = os.path.basename(data)
            dataset_type = "train" if is_training else "test_or_val"
            output_filename = f"enhanced_{dataset_type}_{base_filename}"
    else:
        df = data.copy()
        # Default filename if needed
        if output_filename is None:
            dataset_type = "train" if is_training else "test_or_val"
            output_filename = f"{dataset_type}_enhanced_scaled_data.csv"
    
    print(f"Data shape: {df.shape}")
    print(f"Missing values before preprocessing: {df.isna().sum().sum()}")
    
    # Define columns that we don't want to impute or scale
    id_columns = ['PatientID', 'Hours']
    categorical_columns = ['Gender', 'ICUType']
    
    # If training data and no preprocessing params provided, fit new ones
    if is_training and preprocessing_params is None:
        preprocessing_params = fit_enhanced_preprocessor(df, output_folder)
        scaling_info = preprocessing_params['scaling_info']
        feature_columns = scaling_info['feature_columns']
        stats = preprocessing_params['stats']
        categorical_modes = preprocessing_params['categorical_modes']
    elif preprocessing_params is None:
        # Load preprocessing parameters
        robust_scaler = pickle.load(open(os.path.join(output_folder, 'robust_scaler.pkl'), 'rb'))
        minmax_scaler = pickle.load(open(os.path.join(output_folder, 'minmax_scaler.pkl'), 'rb'))
        scaling_info = pickle.load(open(os.path.join(output_folder, 'scaling_info.pkl'), 'rb'))
        stats = pickle.load(open(os.path.join(output_folder, 'improved_imputation_stats.pkl'), 'rb'))
        categorical_modes = pickle.load(open(os.path.join(output_folder, 'improved_categorical_modes.pkl'), 'rb'))
        
        preprocessing_params = {
            'robust_scaler': robust_scaler,
            'minmax_scaler': minmax_scaler,
            'scaling_info': scaling_info,
            'stats': stats,
            'categorical_modes': categorical_modes
        }
        feature_columns = scaling_info['feature_columns']
    else:
        scaling_info = preprocessing_params['scaling_info']
        feature_columns = scaling_info['feature_columns']
        stats = preprocessing_params['stats']
        categorical_modes = preprocessing_params['categorical_modes']
    
    # Preprocess without scaling using normal distribution sampling
    df_processed = improved_preprocess_without_scaling(
        df, 
        feature_columns, 
        id_columns, 
        categorical_columns, 
        preprocessing_params['categorical_modes'],
        stats=preprocessing_params['stats'],
        random_state=random_state
    )
    
    print(f"Missing values after imputation: {df_processed[feature_columns].isna().sum().sum()}")
    
    # Apply enhanced scaling
    print("Applying enhanced feature scaling...")
    df_scaled = apply_enhanced_scaling(df_processed, preprocessing_params)
    
    # Save the preprocessed data if output folder is provided
    output_path = None
    if output_folder:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            
        output_path = os.path.join(output_folder, output_filename)
        df_scaled.to_csv(output_path, index=False)
        print(f"Enhanced preprocessed data saved to: {output_path}")
    
    # Print some statistics about the preprocessing
    print("\nPreprocessing Summary:")
    print(f"Total rows: {len(df_scaled)}")
    print(f"Total features: {len(feature_columns)}")
    print(f"Features with log transform: {len(scaling_info['log_features'])}")
    print(f"Features with robust scaling: {len(scaling_info['robust_features'])}")
    print(f"Features with minmax scaling: {len(scaling_info['minmax_features'])}")
    print(f"Missing values in original data: {df.isna().sum().sum()}")
    print(f"Missing values in preprocessed data: {df_scaled.isna().sum().sum()}")
    
    if is_training:
        return df_scaled, output_path, preprocessing_params
    else:
        return df_scaled, output_path


In [25]:
train_data_path = "/home/taekim/csv_data/patient_set-a.csv"  # Training set
val_data_path = "/home/taekim/csv_data/patient_set-b.csv"    # Validation set
test_data_path = "/home/taekim/csv_data/patient_set-c.csv"   # Test set
output_folder = "enhanced_preprocessed_data"
# Reload your data
train_data = pd.read_csv(train_data_path)
val_data = pd.read_csv(val_data_path)
test_data = pd.read_csv(test_data_path)

# Load preprocessing parameters
with open(os.path.join(output_folder, 'scaling_info.pkl'), 'rb') as f:
    scaling_info = pickle.load(f)
with open(os.path.join(output_folder, 'improved_imputation_stats.pkl'), 'rb') as f:
    stats = pickle.load(f)
with open(os.path.join(output_folder, 'improved_categorical_modes.pkl'), 'rb') as f:
    categorical_modes = pickle.load(f)

# Define columns
id_columns = ['PatientID', 'Hours']
categorical_columns = ['Gender', 'ICUType']
feature_columns = scaling_info['feature_columns']

# Apply only imputation, not scaling
train_imputed_only = improved_preprocess_without_scaling(
    train_data, feature_columns, id_columns, categorical_columns, 
    categorical_modes, stats=stats)

val_imputed_only = improved_preprocess_without_scaling(
    val_data, feature_columns, id_columns, categorical_columns, 
    categorical_modes, stats=stats)

test_imputed_only = improved_preprocess_without_scaling(
    test_data, feature_columns, id_columns, categorical_columns, 
    categorical_modes, stats=stats)

# Save to new files
train_imputed_only.to_csv(os.path.join(output_folder, 'imputed_only_set-a.csv'), index=False)
val_imputed_only.to_csv(os.path.join(output_folder, 'imputed_only_set-b.csv'), index=False)
test_imputed_only.to_csv(os.path.join(output_folder, 'imputed_only_set-c.csv'), index=False)

/tmp/ipykernel_356242/1759002329.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].fillna(categorical_modes[col], inplace=True)


Applying forward filling...


KeyboardInterrupt: 

In [ ]:
'''
import numpy as np
import pandas as pd
import os
import pickle
from sklearn.preprocessing import RobustScaler, MinMaxScaler

# Define data paths
train_data_path = "/home/taekim/csv_data/patient_set-a.csv"  # Training set
val_data_path = "/home/taekim/csv_data/patient_set-b.csv"    # Validation set
test_data_path = "/home/taekim/csv_data/patient_set-c.csv"   # Test set
output_folder = "enhanced_preprocessed_data"

# Process training data and get preprocessing parameters
train_enhanced, train_enhanced_path, preprocessing_params = enhanced_preprocess_patient_data(
    train_data_path,
    output_folder=output_folder,
    output_filename="enhanced_sampled_set-a.csv",
    is_training=True
)

# Process validation data using same parameters
val_enhanced, val_enhanced_path = enhanced_preprocess_patient_data(
    val_data_path,
    preprocessing_params=preprocessing_params,
    output_folder=output_folder,
    output_filename="enhanced_sampled_set-b.csv",
    is_training=False
)

# Process test data using same parameters
test_enhanced, test_enhanced_path = enhanced_preprocess_patient_data(
    test_data_path,
    preprocessing_params=preprocessing_params,
    output_folder=output_folder,
    output_filename="enhanced_sampled_set-c.csv",
    is_training=False
)

print("\nAll datasets have been preprocessed with enhanced method:")
print(f"Training data: {train_enhanced_path}")
print(f"Validation data: {val_enhanced_path}")
print(f"Test data: {test_enhanced_path}")
'''

Data shape: (196000, 43)
Missing values before preprocessing: 5904543
Training data shape: (196000, 43)
Missing values before preprocessing: 5904543
Log transform features: 12
MinMax features: 4
Robust scale features: 23


/tmp/ipykernel_333586/2423481867.py:62: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].fillna(categorical_modes.get(col, df_processed[col].mode()[0]), inplace=True)
/tmp/ipykernel_333586/2423481867.py:76: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

Applying forward filling...
Missing values after forward filling: 2784218
Sampling from normal distribution for remaining NAs...
Sampled 27679 values for BUN from N(25.47, 20.98)
Sampled 27624 values for Creatinine from N(1.37, 1.43)
Sampled 75136 values for DiasABP from N(59.06, 12.85)
Sampled 79351 values for FiO2 from N(0.55, 0.16)
Sampled 14130 values for GCS from N(11.59, 4.00)
Sampled 37204 values for Glucose from N(136.76, 55.66)
Sampled 29433 values for HCO3 from N(23.59, 4.34)
Sampled 25931 values for HCT from N(31.62, 5.06)
Sampled 11353 values for HR from N(86.72, 17.64)
Sampled 92806 values for Height from N(169.79, 14.64)
Sampled 34689 values for K from N(4.14, 0.61)
Sampled 75678 values for MAP from N(81.41, 16.95)
Sampled 85403 values for MechVent from N(1.00, 0.00)
Sampled 36436 values for Mg from N(2.02, 0.40)
Sampled 48976 values for NIDiasABP from N(57.19, 14.05)
Sampled 49125 values for NIMAP from N(76.12, 14.26)
Sampled 48557 values for NISysABP from N(116.65, 23.4

/tmp/ipykernel_333586/1759002329.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].fillna(categorical_modes[col], inplace=True)


Applying forward filling...
Missing values after forward filling: 2784218
Sampling from normal distribution for remaining NAs...
Sampled 27679 values for BUN from N(25.47, 20.98)
Sampled 27624 values for Creatinine from N(1.37, 1.43)
Sampled 75136 values for DiasABP from N(59.06, 12.85)
Sampled 79351 values for FiO2 from N(0.55, 0.16)
Sampled 14130 values for GCS from N(11.59, 4.00)
Sampled 37204 values for Glucose from N(136.76, 55.66)
Sampled 29433 values for HCO3 from N(23.59, 4.34)
Sampled 25931 values for HCT from N(31.62, 5.06)
Sampled 11353 values for HR from N(86.72, 17.64)
Sampled 92806 values for Height from N(169.79, 14.64)
Sampled 34689 values for K from N(4.14, 0.61)
Sampled 75678 values for MAP from N(81.41, 16.95)
Sampled 85403 values for MechVent from N(1.00, 0.00)
Sampled 36436 values for Mg from N(2.02, 0.40)
Sampled 48976 values for NIDiasABP from N(57.19, 14.05)
Sampled 49125 values for NIMAP from N(76.12, 14.26)
Sampled 48557 values for NISysABP from N(116.65, 23.4

/tmp/ipykernel_333586/1759002329.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].fillna(categorical_modes[col], inplace=True)


Missing values after forward filling: 2766959
Sampling from normal distribution for remaining NAs...
Sampled 27312 values for BUN from N(25.47, 20.98)
Sampled 27191 values for Creatinine from N(1.37, 1.43)
Sampled 73825 values for DiasABP from N(59.06, 12.85)
Sampled 80651 values for FiO2 from N(0.55, 0.16)
Sampled 14003 values for GCS from N(11.59, 4.00)
Sampled 35655 values for Glucose from N(136.76, 55.66)
Sampled 28794 values for HCO3 from N(23.59, 4.34)
Sampled 25161 values for HCT from N(31.62, 5.06)
Sampled 11226 values for HR from N(86.72, 17.64)
Sampled 94570 values for Height from N(169.79, 14.64)
Sampled 33083 values for K from N(4.14, 0.61)
Sampled 74225 values for MAP from N(81.41, 16.95)
Sampled 86873 values for MechVent from N(1.00, 0.00)
Sampled 36164 values for Mg from N(2.02, 0.40)
Sampled 47023 values for NIDiasABP from N(57.19, 14.05)
Sampled 47066 values for NIMAP from N(76.12, 14.26)
Sampled 46354 values for NISysABP from N(116.65, 23.43)
Sampled 32848 values for 

/tmp/ipykernel_333586/1759002329.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].fillna(categorical_modes[col], inplace=True)


Missing values after forward filling: 2773743
Sampling from normal distribution for remaining NAs...
Sampled 28042 values for BUN from N(25.47, 20.98)
Sampled 28010 values for Creatinine from N(1.37, 1.43)
Sampled 73354 values for DiasABP from N(59.06, 12.85)
Sampled 80787 values for FiO2 from N(0.55, 0.16)
Sampled 13659 values for GCS from N(11.59, 4.00)
Sampled 37084 values for Glucose from N(136.76, 55.66)
Sampled 29791 values for HCO3 from N(23.59, 4.34)
Sampled 26099 values for HCT from N(31.62, 5.06)
Sampled 11037 values for HR from N(86.72, 17.64)
Sampled 93198 values for Height from N(169.79, 14.64)
Sampled 34274 values for K from N(4.14, 0.61)
Sampled 73800 values for MAP from N(81.41, 16.95)
Sampled 85642 values for MechVent from N(1.00, 0.00)
Sampled 37276 values for Mg from N(2.02, 0.40)
Sampled 51046 values for NIDiasABP from N(57.19, 14.05)
Sampled 51184 values for NIMAP from N(76.12, 14.26)
Sampled 50463 values for NISysABP from N(116.65, 23.43)
Sampled 34337 values for 